In [ ]:
!git clone https://github.com/gregdurrett/fp-dataset-artifacts.git
!pip3 install -r fp-dataset-artifacts/requirements.txt
!pip3 install datasets
!pip3 install spacy nltk
!python -m spacy download en_core_web_sm
!pip3 install wordfreq

import spacy
from nltk.corpus import wordnet
from datasets import load_dataset
import random
import nltk
from transformers import pipeline
import torch
nltk.download('wordnet')
from wordfreq import word_frequency
from nltk import pos_tag, word_tokenize
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
import torch
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from collections import Counter

Cloning into 'fp-dataset-artifacts'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (36/36), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 36 (delta 12), reused 33 (delta 12), pack-reused 0 (from 0)
Receiving objects: 100% (36/36), 13.95 KiB | 1.99 MiB/s, done.
Resolving deltas: 100% (12/12), done.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that ar

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!python3 fp-dataset-artifacts/run.py --do_train --task nli --dataset snli --output_dir ./trained_model/
!python3 run.py --do_eval --task nli --dataset snli --model ./trained_model/ --output_dir ./eval_output/

Traceback (most recent call last):
  File "/content/fp-dataset-artifacts/run.py", line 2, in <module>
    from transformers import AutoTokenizer, AutoModelForSequenceClassification, \
  File "/usr/local/lib/python3.10/dist-packages/transformers/__init__.py", line 26, in <module>
    from . import dependency_versions_check
  File "/usr/local/lib/python3.10/dist-packages/transformers/dependency_versions_check.py", line 16, in <module>
    from .utils.versions import require_version, require_version_core
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/__init__.py", line 34, in <module>
    from .generic import (
  File "/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py", line 462, in <module>
    import torch.utils._pytree as _torch_pytree
  File "/usr/local/lib/python3.10/dist-packages/torch/__init__.py", line 290, in <module>
    from torch._C import *  # noqa: F403
KeyboardInterrupt
^C


In [ ]:
dataset = load_dataset('snli', split='test')

set_1 = dataset.select(range(len(dataset)))
set_2 = dataset.select(range(len(dataset)))
set_3 = dataset.select(range(len(dataset)))
set_4 = dataset.select(range(len(dataset)))
set_5 = dataset.select(range(len(dataset)))

In [ ]:
device = 0 if torch.cuda.is_available() else -1

nli_model = pipeline("text-classification", model="facebook/bart-large-mnli", device=device)

# check if the sentence contains 'is' or 'are'
def contains_is_or_are(example):
    return ' is ' in example['hypothesis'] or ' are ' in example['hypothesis']

# function to introduce negation
def introduce_negation_with_nli(example):
    original_label = example['label']
    original_hypothesis = example['hypothesis']

    # modify hypothesis if it contains 'is'
    if ' is ' in original_hypothesis:
        negated_hypothesis = original_hypothesis.replace(' is ', ' is not ')
    elif ' are ' in original_hypothesis:
        negated_hypothesis = original_hypothesis.replace(' are ', ' are not ')
    else:
      print("here")

    premise = example['premise']
    result = nli_model(f"premise: {premise} hypothesis: {negated_hypothesis}")

    # check if the negation changes the relation to entailment, contradiction, or neutral
    new_label = result[0]['label']

    # entailment -> 0, contradiction -> 2, neutral -> 1
    label_mapping = {'entailment': 0, 'contradiction': 2, 'neutral': 1}
    example['label'] = label_mapping.get(new_label)

    example['hypothesis'] = negated_hypothesis

    return example

hypotheses_with_is = set_1.filter(contains_is_or_are)

negated_hypotheses = hypotheses_with_is.map(introduce_negation_with_nli)

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6215 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
for i in range(10):
    print(f"Original #{i+1}:")
    print(f"Premise: {hypotheses_with_is['premise'][i]}")
    print(f"Hypothesis: {hypotheses_with_is['hypothesis'][i]}")
    print(f"Label: {hypotheses_with_is['label'][i]}")

    print(f"Negated #{i+1}:")
    print(f"Premise: {negated_hypotheses['premise'][i]}")
    print(f"Hypothesis: {negated_hypotheses['hypothesis'][i]}")
    print(f"Label: {negated_hypotheses['label'][i]}")

    print("-" * 40)

Original #1:
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition.
Label: 1
Negated #1:
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is not training his horse for a competition.
Label: 2
----------------------------------------
Original #2:
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is at a diner, ordering an omelette.
Label: 2
Negated #2:
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is not at a diner, ordering an omelette.
Label: 2
----------------------------------------
Original #3:
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is outdoors, on a horse.
Label: 0
Negated #3:
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is not outdoors, on a horse.
Label: 2
----------------------------------------
Original #4:
Premise

In [ ]:
# switch one character with its neighbor, avoiding spaces
def switch_random_neighbor(sentence):
    sentence_list = list(sentence)

    valid_indices = [
        i for i in range(len(sentence) - 1)
        if sentence_list[i].isalnum() and sentence_list[i + 1].isalnum()
    ]

    # if no valid indices are found, return original sentence
    if not valid_indices:
        return sentence

    # keep trying until we find a valid index where neither the character nor its neighbor is a space
    while True:
        index = random.randint(0, len(sentence) - 2)
        if (sentence_list[index].isalnum() and sentence_list[index + 1].isalnum()):
            break  # Found a valid position to swap

    # swap the selected character with its neighbor
    sentence_list[index], sentence_list[index + 1] = sentence_list[index + 1], sentence_list[index]

    # convert the list back to a string and return the modified sentence
    return ''.join(sentence_list)

def modify_hypothesis(example):
    example['hypothesis'] = switch_random_neighbor(example['hypothesis'])
    return example

modified_set_2 = set_2.map(modify_hypothesis)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
for i in range(10):
    print(f"Original #{i+1}:")
    print(f"Premise: {modified_set_2['premise'][i]}")
    print(f"Hypothesis: {modified_set_2['hypothesis'][i]}")
    print(f"Label: {modified_set_2['label'][i]}")

    print(f"Negated #{i+1}:")
    print(f"Premise: {set_2['premise'][i]}")
    print(f"Hypothesis: {set_2['hypothesis'][i]}")
    print(f"Label: {set_2['label'][i]}")

    print("-" * 40)

Original #1:
Premise: a man in a white and green shirt holding a purple umbrella.
Hypothesis: A man hloding a umbrella.
Label: 0
Negated #1:
Premise: a man in a white and green shirt holding a purple umbrella.
Hypothesis: A man holding a umbrella.
Label: 0
----------------------------------------
Original #2:
Premise: a man in a white and green shirt holding a purple umbrella.
Hypothesis: A man punching a rbick wall.
Label: 2
Negated #2:
Premise: a man in a white and green shirt holding a purple umbrella.
Hypothesis: A man punching a brick wall.
Label: 2
----------------------------------------
Original #3:
Premise: A tourist is taking pictures of a cyclist with an umbrella.
Hypothesis: A cyclist rides through the arin with an umbrella.
Label: 1
Negated #3:
Premise: A tourist is taking pictures of a cyclist with an umbrella.
Hypothesis: A cyclist rides through the rain with an umbrella.
Label: 1
----------------------------------------
Original #4:
Premise: A tourist is taking pictures

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model_mlm = BertForMaskedLM.from_pretrained('bert-base-uncased').to(device) 
model_bert = BertModel.from_pretrained('bert-base-uncased').to(device) 

def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt').to(device) 
    with torch.no_grad():
        outputs = model_bert(**inputs)
    return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

# replace a word in a sentence using BERT
def replace_word_with_bert(sentence, word_to_replace):
    masked_sentence = sentence.replace(word_to_replace, '[MASK]', 1)
    inputs = tokenizer(masked_sentence, return_tensors='pt').to(device) 
    mask_token_index = torch.where(inputs['input_ids'] == tokenizer.mask_token_id)[1]

    with torch.no_grad():
        logits = model_mlm(**inputs).logits

    mask_token_logits = logits[0, mask_token_index, :]
    top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

    # get the top replacement word (that is not the og word)
    for token in top_5_tokens:
        replacement_word = tokenizer.decode([token])
        if replacement_word.lower() != word_to_replace.lower():
            return replacement_word
    return word_to_replace

# replace a word and calculate similarity 
def replace_word_and_get_similarity(sentence, word_to_replace):
    replaced_word = replace_word_with_bert(sentence, word_to_replace)
    replaced_sentence = sentence.replace(word_to_replace, replaced_word, 1)

    # get embeddings for og and replaced sentences
    original_embedding = get_sentence_embedding(sentence)
    replaced_embedding = get_sentence_embedding(replaced_sentence)

    # calc cosine similarity 
    similarity = cosine_similarity(original_embedding, replaced_embedding)[0][0]
    return similarity, replaced_word, replaced_sentence

# find the word whose replacement least changes the meaning
def find_least_changed_word(sentence):
    words = word_tokenize(sentence)
    pos_tags = pos_tag(words)

    candidates = [word for word, pos in pos_tags if pos in ['NN', 'NNS', 'JJ']]

    best_similarity = -1
    best_word = None
    best_replaced_word = None
    best_replaced_sentence = None

    for word in candidates:
        similarity, replaced_word, replaced_sentence = replace_word_and_get_similarity(sentence, word)
        if similarity > best_similarity:
            best_similarity = similarity
            best_word = word
            best_replaced_word = replaced_word
            best_replaced_sentence = replaced_sentence

    return best_word, best_replaced_word, best_replaced_sentence, best_similarity

def replace_in_dataset(row):
    sentence = row['hypothesis']
    _, _, new_sentence, _ = find_least_changed_word(sentence)

    row['hypothesis'] = new_sentence
    return row

set_3_with_synonyms = set_3.map(replace_in_dataset)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
for i in range(10):
    print(f"Original #{i+1}:")
    print(f"Premise: {set_3_with_synonyms['premise'][i]}")
    print(f"Hypothesis: {set_3_with_synonyms['hypothesis'][i]}")
    print(f"Label: {set_3_with_synonyms['label'][i]}")

    print(f"Negated #{i+1}:")
    print(f"Premise: {set_3['premise'][i]}")
    print(f"Hypothesis: {set_3['hypothesis'][i]}")
    print(f"Label: {set_3['label'][i]}")

    print("-" * 40) 

Original #1:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a rest from a hike.
Hypothesis: Everyone decide to go to a restaurant for lunch.
Label: 2
Negated #1:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a rest from a hike.
Hypothesis: Everyone decide to go to a restaurant for dinner.
Label: 2
----------------------------------------
Original #2:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a rest from a hike.
Hypothesis: Three guys are taking a break.
Label: 1
Negated #2:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a rest from a hike.
Hypothesis: Three people are taking a break.
Label: 1
----------------------------------------
Original #3:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a rest f

In [ ]:
# get antonyms of a word using WordNet
def get_antonyms(word):
    antonyms = []
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            if lemma.antonyms():
                antonyms.append(lemma.antonyms()[0].name())
    return list(set(antonyms))  # Return unique antonyms

# get BERT sentence embeddings
def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = model_bert(**inputs)
    return outputs.last_hidden_state.mean(dim=1).numpy()

# replace a word with its antonym and calculate similarity
def replace_with_antonym_and_get_similarity(sentence, word_to_replace, original_pos):
    antonyms = get_antonyms(word_to_replace)
    if not antonyms:  # if no antonyms found, return og sentence
        print(sentence)
        return sentence, None, 1.0 

    # find the antonym that makes the sentence most different
    best_similarity = 1.0
    best_antonym = None
    best_sentence = sentence

    for antonym in antonyms:
        # ensure the antonym fits the POS
        antonym_pos = pos_tag([antonym])[0][1]
        if antonym_pos != original_pos:
            continue

        replaced_sentence = sentence.replace(word_to_replace, antonym, 1)

        original_embedding = get_sentence_embedding(sentence)
        replaced_embedding = get_sentence_embedding(replaced_sentence)

        similarity = cosine_similarity(original_embedding, replaced_embedding)[0][0]

        if similarity < best_similarity:
            best_similarity = similarity
            best_antonym = antonym
            best_sentence = replaced_sentence

    return best_sentence, best_antonym, best_similarity

# find the word whose antonym changes the meaning the most but fits the context
def find_most_changed_word_with_context(sentence):
    words = word_tokenize(sentence)
    pos_tags = pos_tag(words)

    candidates = [(word, pos) for word, pos in pos_tags if pos in ['NN', 'NNS', 'JJ']]

    best_similarity = 1.0  
    best_word = None
    best_antonym = None
    best_replaced_sentence = sentence

    # for each candidate word, replace it with antonym and calculate similarity
    for word, pos in candidates:
        replaced_sentence, antonym, similarity = replace_with_antonym_and_get_similarity(sentence, word, pos)
        if antonym and similarity < best_similarity:
            best_similarity = similarity
            best_word = word
            best_antonym = antonym
            best_replaced_sentence = replaced_sentence

    return best_word, best_antonym, best_replaced_sentence, best_similarity

# apply the antonym replacement in the dataset
def replace_in_dataset_with_contextual_antonym(row):
    sentence = row['hypothesis']
    _, antonym, new_sentence, _ = find_most_changed_word_with_context(sentence)

    if antonym:
        row['hypothesis'] = new_sentence
    return row

set_4_copy = set_4.select(range(20, 50))

set_4_with_antonyms = set_4_copy.map(replace_in_dataset_with_contextual_antonym)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Students are in class.
Students are in class.
A man is juggling for a crowd.
The person is part of a road crew, fixing a bridge that has started to crumble.
The person is part of a road crew, fixing a bridge that has started to crumble.
The person is part of a road crew, fixing a bridge that has started to crumble.
The person is part of a road crew, fixing a bridge that has started to crumble.
The person is holding a sign that protests high taxes.
The person is holding a sign that protests high taxes.
The person is holding a sign that protests high taxes.
There is one person in this picture, and they are working.
There is one person in this picture, and they are working.
All the vehicles are empty.
A plaza is empty.
No one is watching the person.
No one is watching the person.
People are watching a car in its garage.
People are watching a car in its garage.
People are watching a car in its garage.
A criminal is sitting in a strange car.
A criminal is sitting in a strange car.
There is 

In [ ]:
dataset = load_dataset('snli', split='train')

verb_counter = Counter()

# extract and count verbs in a sentence
def count_verbs_in_sentence(sentence):
    # tokenize the sentence into words
    words = word_tokenize(sentence)

    # perform POS tagging to get parts of speech for each word
    pos_tags = pos_tag(words)
    verbs = [word for word, pos in pos_tags if pos.startswith('VB')]
    verb_counter.update(verbs)

# go through each example in the dataset and count verbs
for example in dataset:
    hypothesis = example['hypothesis']
    premise = example['premise']

    # count verbs in both the premise and hypothesis
    count_verbs_in_sentence(hypothesis)
    count_verbs_in_sentence(premise)

# get the 100 most common verbs
most_common_verbs = verb_counter.most_common(250)


In [ ]:
from nltk.stem import WordNetLemmatizer
nlp = spacy.load('en_core_web_sm')
lemmatizer = WordNetLemmatizer()
device = 0 if torch.cuda.is_available() else -1
nli_model = pipeline("text-classification", model="facebook/bart-large-mnli", device=device)
words_with_antonyms = {
    "man": "woman",
    "woman": "man",
    "men": "women",
    "girl": "boy",
    "boy": "girl",
    "dog": "cat",
    "front": "back",
    "women": "men",
    "child": "adult",
    "beach": "mountain",
    "girls": "boys",
    "dogs": "cats",
    "crowd": "individual",
    "background": "foreground",
    "game": "work",
    "orange": "blue",
    "lady": "gentleman",
    "blue": "orange",
    "city": "countryside",
    "couple": "single",
    "dress": "undress",
    "top": "bottom",
    "baby": "adult",
    "outdoors": "indoors",
    "hand": "foot",
    "kids": "adults",
    "day": "night",
    "ground": "sky",
    "head": "tail",
    "face": "back",
    "team": "individual",
    "construction": "destruction",
    "blond": "brunette",
    "workers": "bosses",
    "dirt": "cleanliness",
    "mountain": "valley",
    "home": "away",
    "back": "front",
    "female": "male",
    "night": "day",
    "cat": "dog",
    "mother": "father",
    "toddler": "adult",
    "sun": "moon",
    "brown": "white",
    "outside": "inside",
    "ladies": "gentlemen",
    "adults": "children",
    "women": "men",
    "sky": "ground",
    "indoors": "outdoors",
    "walk": "run"
}

noun_antonyms = {word: antonym for word, antonym in words_with_antonyms.items() if pos_tag([word])[0][1] in ['NN', 'NNS']}
verb_antonyms = {word: antonym for word, antonym in words_with_antonyms.items() if pos_tag([word])[0][1].startswith('VB')}

# get the correct verb form based on the original verb
def get_correct_verb_form(original_verb, antonym_base):
    doc = nlp(original_verb)
    token = doc[0]
    if token.tag_ == "VBD":  
        return lemmatizer.lemmatize(antonym_base, 'v') + "ed"
    elif token.tag_ == "VBG":  
        return lemmatizer.lemmatize(antonym_base, 'v') + "ing"
    elif token.tag_ == "VBZ": 
        return lemmatizer.lemmatize(antonym_base, 'v') + "s"
    return antonym_base  

# replace only the first noun or verb with its antonym
def replace_first_antonym_in_hypothesis(row):
    sentence = row['hypothesis']
    words = word_tokenize(sentence)
    pos_tags = pos_tag(words)

    new_sentence = []
    replaced = False 

    for word, pos in pos_tags:
        if not replaced:
            # if the word is a noun, replace with antonym
            if pos.startswith('NN') and word in noun_antonyms:
                new_sentence.append(noun_antonyms[word])
                replaced = True
            # if the word is a verb, replace with antonym after lemmatization
            elif pos.startswith('VB'):
                base_form = lemmatizer.lemmatize(word, 'v')
                if base_form in verb_antonyms:
                    antonym_base = verb_antonyms[base_form]
                    new_sentence.append(get_correct_verb_form(word, antonym_base))
                    replaced = True
                else:
                    new_sentence.append(word)
            else:
                new_sentence.append(word)
        else:
            new_sentence.append(word) 

    row['hypothesis'] = " ".join(new_sentence)
    return row

# check if NLI model changes the label after antonym replacement
def check_label_change_with_nli(row):
    original_premise = row['premise']
    original_hypothesis = row['hypothesis']

    # get the NLI prediction on the premise and hypothesis
    result = nli_model(f"premise: {original_premise} hypothesis: {original_hypothesis}")

    # entailment -> 0, contradiction -> 2, neutral -> 1
    label_mapping = {'entailment': 0, 'contradiction': 2, 'neutral': 1}
    new_label = label_mapping.get(result[0]['label'], row['label'])  

    row['label'] = new_label
    return row

set_4 = set_4.map(replace_first_antonym_in_hypothesis)

set_4 = set_4.map(check_label_change_with_nli)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
for i in range(10):
    print(f"Original #{i+1}:")
    print(f"Premise: {set_4_with_antonyms['premise'][i]}")
    print(f"Hypothesis: {set_4_with_antonyms['hypothesis'][i]}")
    print(f"Label: {set_4_with_antonyms['label'][i]}")

    print(f"Negated #{i+1}:")
    print(f"Premise: {set_3['premise'][i]}")
    print(f"Hypothesis: {set_3['hypothesis'][i]}")
    print(f"Label: {set_3['label'][i]}")

    print("-" * 40)

Original #1:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a rest from a hike.
Hypothesis: Everyone decide to go to a restaurant for dinner .
Label: 2
Negated #1:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a rest from a hike.
Hypothesis: Everyone decide to go to a restaurant for dinner.
Label: 2
----------------------------------------
Original #2:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a rest from a hike.
Hypothesis: Three people are taking a break .
Label: 0
Negated #2:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a rest from a hike.
Hypothesis: Three people are taking a break.
Label: 1
----------------------------------------
Original #3:
Premise: Three people, two males and a female, are sitting in front of a tall tree on the ground during a r

In [ ]:
neutral_phrases = [
    ", and the sky is blue.",
    ", and the grass is green.",
    ", and the sun is shining.",
    ", and the weather is nice.",
    ", and the birds are singing.",
    ", and the clouds are white.",
    ", and the ocean is calm.",
    ", and the stars are visible.",
    ", and the wind is gentle.",
    ", and the trees are tall.",
    ", and the air is fresh.",
    ", and the water is clear.",
    ", and the flowers are blooming.",
    ", and the moon is bright.",
    ", and the mountains are high.",
    ", and the leaves are falling.",
    ", and the roads are quiet.",
    ", and the lake is still.",
    ", and the horizon is wide.",
    ", and the sun is setting.",
    ", and the river is flowing.",
    ", and the breeze is cool.",
    ", and the day is calm.",
    ", and the sky is clear.",
    ", and the stars are twinkling.",
    ", and the clouds are drifting.",
    ", and the fields are vast.",
    ", and the forest is quiet.",
    ", and the hills are rolling.",
    ", and the breeze is soft.",
    ", and the ground is firm.",
    ", and the air is still.",
    ", and the rain is light.",
    ", and the sky is vast.",
    ", and the sand is warm.",
    ", and the valley is deep.",
    ", and the mountains are distant.",
    ", and the waves are gentle.",
    ", and the light is soft.",
    ", and the snow is falling.",
    ", and the city is peaceful.",
    ", and the stars are bright.",
    ", and the river is calm.",
    ", and the fog is lifting.",
    ", and the path is clear.",
    ", and the meadows are green.",
    ", and the moon is rising.",
    ", and the air is crisp.",
    ", and the streets are quiet.",
    ", and the forest is serene.",
    ", and the view is stunning.",
    ", and the sun is warm.",
    ", and the night is quiet.",
    ", and the hills are silent.",
    ", and the rain is steady.",
    ", and the air is cool.",
    ", and the park is empty.",
    ", and the sun is low.",
    ", and the fields are quiet.",
    ", and the light is fading.",
    ", and the ocean is vast.",
    ", and the stars are steady.",
    ", and the beach is wide.",
    ", and the town is still.",
    ", and the night is peaceful.",
    ", and the horizon is clear.",
    ", and the water is calm.",
    ", and the shadows are long.",
    ", and the landscape is vast.",
    ", and the garden is blooming.",
    ", and the road is long.",
    ", and the wind is calm.",
    ", and the lake is quiet.",
    ", and the hills are green."

]

# randomly add a neutral phrase to a hypothesis
def add_random_neutral_phrase(hypothesis):
    # remove the period at the end if it exists
    if hypothesis.endswith('.'):
        hypothesis = hypothesis[:-1]  

    random_phrase = random.choice(neutral_phrases)

    # append a neutral phrase to the hypothesis
    modified_hypothesis = hypothesis + random_phrase

    return modified_hypothesis

def add_neutral_to_hypothesis(example):
    example['hypothesis'] = add_random_neutral_phrase(example['hypothesis'])
    return example

set_5_with_neutral_phrases = set_5.map(add_neutral_to_hypothesis)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
for i in range(10):
    print(f"Original #{i+1}:")
    print(f"Premise: {set_5_with_neutral_phrases['premise'][i]}")
    print(f"Hypothesis: {set_5_with_neutral_phrases['hypothesis'][i]}")
    print(f"Label: {set_5_with_neutral_phrases['label'][i]}")

    print(f"Negated #{i+1}:")
    print(f"Premise: {set_5['premise'][i]}")
    print(f"Hypothesis: {set_5['hypothesis'][i]}")
    print(f"Label: {set_5['label'][i]}")

    print("-" * 40) 

Original #1:
Premise: A redheaded young boy in front of a fence holds a fishing pole in one hand and a line with a fish at the end of it in the other.
Hypothesis: A redheaded young boy is in front of a car, and the river is flowing.
Label: 2
Negated #1:
Premise: A redheaded young boy in front of a fence holds a fishing pole in one hand and a line with a fish at the end of it in the other.
Hypothesis: A redheaded young boy is in front of a car.
Label: 2
----------------------------------------
Original #2:
Premise: A redheaded young boy in front of a fence holds a fishing pole in one hand and a line with a fish at the end of it in the other.
Hypothesis: A boy holds his fishing pole, and the wind is gentle.
Label: 0
Negated #2:
Premise: A redheaded young boy in front of a fence holds a fishing pole in one hand and a line with a fish at the end of it in the other.
Hypothesis: A boy holds his fishing pole.
Label: 0
----------------------------------------
Original #3:
Premise: A redheaded 

In [ ]:
set_6 = dataset.select(range(len(dataset)))
set_7 = dataset.select(range(len(dataset)))

In [ ]:
def expand_contractions(sentence):
    contractions = {
        "doesn't": "does not",
        "don't": "do not",
        "can't": "cannot",
        "won't": "will not",
        "isn't": "is not",
        "aren't": "are not",
        "wasn't": "was not",
        "weren't": "were not",
        "hasn't": "has not",
        "haven't": "have not",
        "hadn't": "had not",
        "wouldn't": "would not",
        "couldn't": "could not",
        "shouldn't": "should not",
        "mustn't": "must not",
        "shan't": "shall not",
        "he's": "he is",
        "she's": "she is",
        "it's": "it is",
        "that's": "that is",
        "what's": "what is",
        "let's": "let us",
        "who's": "who is",
        "I'll": "I will",
        "you'll": "you will",
        "he'll": "he will",
        "she'll": "she will",
        "they'll": "they will",
        "we'll": "we will",
        "I'd": "I would",
        "you'd": "you would",
        "he'd": "he would",
        "she'd": "she would",
        "we'd": "we would",
        "they'd": "they would",
        "I've": "I have",
        "you've": "you have",
        "we've": "we have",
        "they've": "they have",
        "isn't": "is not",
        "ain't": "is not"
    }

    # iterate over each contraction and replace it in the sentence
    for contraction, expanded_form in contractions.items():
        sentence = sentence.replace(contraction, expanded_form)

    return sentence

def expand_contractions_in_row(row):
    row['hypothesis'] = expand_contractions(row['hypothesis'])
    return row

set_6 = set_6.map(expand_contractions_in_row)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_dataset
from transformers import pipeline

def swap_genders(sentence):
    gender_swaps = {
        "man": "woman", "woman": "man",
        "he": "she", "she": "he",
        "his": "her", "her": "his",
        "him": "her", "her": "him",
        "boy": "girl", "girl": "boy",
        "father": "mother", "mother": "father",
        "son": "daughter", "daughter": "son",
        "brother": "sister", "sister": "brother"
    }

    words = sentence.split()
    swapped_sentence = []

    for word in words:
        # check for the lowercased version of the word in gender_swaps
        lower_word = word.lower()
        if lower_word in gender_swaps:
            # preserve the case: if the original word was capitalized, capitalize the swapped word
            swapped_word = gender_swaps[lower_word]
            if word[0].isupper():
                swapped_word = swapped_word.capitalize()
            swapped_sentence.append(swapped_word)
        else:
            swapped_sentence.append(word)

    return " ".join(swapped_sentence)

nli_model = pipeline("text-classification", model="facebook/bart-large-mnli")

def update_hypothesis_and_label(row):
    premise = row['premise']
    hypothesis = row['hypothesis']

    # swap genders in the hypothesis
    swapped_hypothesis = swap_genders(hypothesis)

    # get the label for the swapped hypothesis using the NLI model
    result = nli_model(f"premise: {premise} hypothesis: {swapped_hypothesis}")
    new_label = result[0]['label']

    row['hypothesis'] = swapped_hypothesis
    row['label'] = new_label

    return row

set_7 = set_7.map(update_hypothesis_and_label)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [ ]:
set_1.to_json("/content/set_1.jsonl")
set_2.to_json("/content/set_2.jsonl")
set_3.to_json("/content/set_3.jsonl")
set_4.to_json("/content/set_4.jsonl")
set_5.to_json("/content/set_5.jsonl")

Creating json from Arrow format:   0%|          | 0/7 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

1511447

In [ ]:
set_6.to_json("/content/set_6.jsonl")
set_7.to_json("/content/set_7.jsonl")

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

1511447

In [ ]:
dataset.to_json("/content/og.jsonl")

Creating json from Arrow format:   0%|          | 0/10 [00:00<?, ?ba/s]

1509349

In [ ]:
!python3 ./fp-dataset-artifacts/run.py --do_eval --task nli --dataset ./og.jsonl --model ./trained_model/ --output_dir ./eval_output/
!python3 ./fp-dataset-artifacts/run.py --do_eval --task nli --dataset ./set_1.jsonl --model ./trained_model/ --output_dir ./eval_output/
!python3 ./fp-dataset-artifacts/run.py --do_eval --task nli --dataset ./set_2.jsonl --model ./trained_model/ --output_dir ./eval_output/
!python3 ./fp-dataset-artifacts/run.py --do_eval --task nli --dataset ./set_3.jsonl --model ./trained_model/ --output_dir ./eval_output/
!python3 ./fp-dataset-artifacts/run.py --do_eval --task nli --dataset ./set_4.jsonl --model ./trained_model/ --output_dir ./eval_output/
!python3 ./fp-dataset-artifacts/run.py --do_eval --task nli --dataset ./set_5.jsonl --model ./trained_model/ --output_dir ./eval_output/
!python3 ./fp-dataset-artifacts/run.py --do_eval --task nli --dataset ./set_6.jsonl --model ./trained_model/ --output_dir ./eval_output/
!python3 ./fp-dataset-artifacts/run.py --do_eval --task nli --dataset ./set_7.jsonl --model ./trained_model/ --output_dir ./eval_output/

2024-09-29 22:55:24.082159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 22:55:24.102748: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 22:55:24.109093: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 22:55:25.290370: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Generating train split: 33010 examples [00:00, 623478.39 examples/s]
Traceback (most recent call last):
  File "/content/./fp-dataset-artifacts/run.py", line 208, in <module>
    main()
  File "/content/./fp-dataset-artifacts/run.py", line 8